In [ ]:
import torch as th

In [ ]:
import matplotlib.pyplot as plt

maxv = 50

# This model simulates a falling body that reaches a maximum speed.
# The non linearity on the maximum speed is intentional. This notebook
# experiments with many models that catches this non-linearity.
class ReferenceModel(th.nn.Module):
  def __init__(self):
    super(ReferenceModel, self).__init__()
    g = -9.8
    dt = 0.016
    self.A = th.tensor([[1, dt], [0, 1]], requires_grad=False).T
    self.b = th.tensor([0, dt * g], requires_grad=False).T
  
  def forward(self, inputs):
    x = inputs @ self.A + self.b
    x[:, 1] = th.clamp(x[:, 1], -maxv, maxv)
    return x

ref_model = ReferenceModel()

In [ ]:
def create_trajectory(model, vel):  
  '''Creates a trajectory with a starting velocity vel'''
  
  with th.no_grad():
    first = th.tensor([[0, vel]], dtype=th.float32)
    X = [first]
    for i in range(800):
      x0 = X[-1]
      X.append(ref_model(x0))

    return th.cat(X)

X = create_trajectory(ref_model, maxv)

def plot_trajectory(X, ax = None):
  if not ax:
    fig, ax = plt.subplots()
  ax.plot(range(len(X)), [0] * len(X), ':', color='#F0F0F0')
  ax.plot(range(len(X)), [-maxv] * len(X), ':', color='#F0F0F0')
  ax.plot(range(len(X)), [x[0] for x in X], color="#8080FF", label='pos')
  ax.plot(range(len(X)), [x[1] for x in X], color='#FF8044', label='vel')
  ax.legend()

plot_trajectory(X)

In [ ]:
from collections.abc import Iterable

def flatten(l):
  r = []
  for i in l:
    if isinstance(i, Iterable):
      r.extend(flatten(i))
    else:
      r.append(i)
  return r

flatten([[1, 2 , [3, 5, 2]], [2]])
# print(type([]))

In [ ]:
Xtrain = []
Ytrain = []
i = 0

fig, axs = plt.subplots(3, 3, figsize=(12, 6))
axs = flatten(axs)

# Creates a data set with several different starting velocities
for v in th.arange(-0.9, 0.901, 0.2):
  v *= maxv
  X = create_trajectory(ref_model, v)
  if i < 9:
    plot_trajectory(X, axs[i])
    i += 1
  Xtrain.append(X[:-1])
  Ytrain.append(X[1:])

Xtrain = th.cat(Xtrain)
Ytrain = th.cat(Ytrain)

print('Xtrain:', Xtrain.shape)
print('Ytrain:', Ytrain.shape)

In [ ]:
def map_reduce(a, kfn, redfn):
  m = {}
  for x in a:
    k = kfn(x)
    elem = m.get(k, None)
    m[k] = redfn(elem, x)
  return m

def red_count(acc, x):
  if acc == None:
    return 1
  return acc + 1

hist_vel = map_reduce(Xtrain, lambda x: int(x[1]), red_count)
hist_pos = map_reduce(Xtrain, lambda x: int(x[1]/4), red_count)

def plot_histogram(hist, label, ax=None):
  if not ax:
    fig, ax = plt.subplots()
  ax.set_title(label + ' count')
  ax.bar(hist.keys(), hist.values())
  ax.set_ylabel('count')
  ax.set_xlabel(label)

fig, axs = plt.subplots(1,2, figsize=(10, 3))
plot_histogram(hist_vel, 'vel', axs[0])
plot_histogram(hist_pos, 'pos/4', axs[1])

In [ ]:
# del models
saved_models = {}

In [ ]:
class Model1(th.nn.Module):
  '''This exploits knowledge from he physical system. Since we know what parameters exists and how they
    affect the system, we can do simple linear regression.'''
  def __init__(self):
    super(Model1, self).__init__()
    self.dt = th.nn.Parameter(th.rand(1))
    self.g = th.nn.Parameter(th.rand(1))
    self.maxv = th.nn.Parameter(th.rand(1))
  
  def forward(self, inputs):
    vel = inputs[:, 1]
    pos = inputs[:, 0]
    pos = vel*self.dt + pos
    vel = self.g*self.dt + vel
    vel = th.clamp(vel, -self.maxv, self.maxv)
    return th.stack((pos, vel), dim=1)


def create_model(mtype, load_state = True):
  model = mtype()
  if load_state and str(mtype) in saved_models:
    saved = saved_models[str(mtype)]
    try:
      model.load_state_dict(saved['state'])
      print('Loaded model state. loss:', saved['loss'])
    except:
      print('Saved state doesn\'t match model')
  return model

print(saved_models.keys())
model1 = create_model(Model1)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return float(param_group['lr'])
  raise Exception('No earning rate found')
      
def fit(model, X=Xtrain, Y=Ytrain, batch_size=None, epochs=1000, optimizer=None, lossfn=th.nn.L1Loss(), patiance = 100, min_lr=1e-9):
  losses = []
  # noimp = 0
  min_loss = 9999999999999999
  best_state = None
  if not optimizer:
    optimizer = th.optim.AdamW(model.parameters(), lr=1e-2)
  lrDecaySch = ReduceLROnPlateau(optimizer, patience=patiance, verbose=True, eps=min_lr*0.1, threshold=1e-6)
  stop = False
  lr = get_lr(optimizer)
  print('Learning rate: ', lr)
  for i in range(epochs):
    if stop:
      break
    optimizer.zero_grad()
    
    if batch_size:
      idx = th.randint(0, len(X), (batch_size,))
      Xtrain = X[idx]
      Ytrain = X[idx]
    else:
      Xtrain = X
      Ytrain = Y
    
    outputs = model(Xtrain)
    
    loss = lossfn(Ytrain, outputs)
    loss.backward()
    optimizer.step()
    
    print('epoch {}, lr: {}, loss {}'.format(i, lr, loss.item()))
    lrDecaySch.step(loss)
    lr = get_lr(optimizer)
    if lr < min_lr:
      stop=True
        
    curr_loss = loss.item()
    losses.append(curr_loss)
    if curr_loss < min_loss:
      # noimp = 0
      min_loss = curr_loss
      best_state = model.state_dict()
    # else:
    #   noimp += 1
    #   if noimp >= patiance:
    #     break
  print('Learning rate: ', lr)
  return losses, min_loss, best_state

import math

def plot_loss(losses):
  fig, ax = plt.subplots(figsize=(10, 2.5))
  ax.plot(range(len(losses)), [math.log(l) for l in losses])


In [ ]:
def save_if_better(mtype, min_loss, state_dict):
  print('minloss', min_loss)
  if str(mtype) in saved_models and min_loss >= saved_models[str(mtype)]['loss']:
    return
  print('Saving better model')
  saved_models[str(mtype)] = {
    'loss': min_loss,
    'state': state_dict
  }

model1 = create_model(Model1)
losses, min_loss, best_state = fit(model1, epochs=2000, optimizer=th.optim.AdamW(model1.parameters(), lr=0.01))
save_if_better(type(model1), min_loss, best_state)
plot_loss(losses)
print(model1.state_dict())

In [ ]:
def compare_plots(ma, mb, vel, ax=None):
  with th.no_grad():
    first = th.tensor([[0, vel]], dtype=th.float32)
    Xa = [first]
    Xb = [first]
    for i in range(1000):
      xa = Xa[-1]
      # [None, :]
      # print(xa.shape)
      Xa.append(ma(Xa[-1]))
      Xb.append(mb(Xb[-1]))
    
    Xa = th.cat(Xa)
    Xb = th.cat(Xb)
    if not ax:
      fig, ax = plt.subplots(figsize=(5, 2.5))
    img = ax.plot(range(len(X)), [0] * len(X), ':', color='#F0F0F0')
    img = ax.plot(range(len(X)), [-maxv] * len(X), ':', color='#F0F0F0')
    def plot(X, style='-'):
      img = ax.plot(range(len(X)), [x[0] for x in X], style, color="#8080FF", label='pos')
      img = ax.plot(range(len(X)), [x[1] for x in X], style, color='#FF8044', label='vel')
    plot(Xa, ':')
    plot(Xb)
    ax.legend()

In [ ]:
def compare_many(ma, mb):
  fig, axs = plt.subplots(3, 3, figsize=(14, 9))
  axs = flatten(axs)

  for i, v in enumerate(th.arange(-0.85, 1.01, 0.23)):
    v *= maxv
    axs[i].set_title(str(v.item()))
    compare_plots(ma, mb, v, axs[i])

compare_many(ref_model, model1)

In [ ]:
class Model2(th.nn.Module):
  '''Model that tries to use tanh as non linearity for speed.
     Since there is no linearity in the position, bypass that with a second linear layer and sum at the end'''
  def __init__(self):
    super(Model2, self).__init__()
    self.lin1 = th.nn.Linear(2, 2)
    self.lin2 = th.nn.Linear(2, 2)
  
  def forward(self, inputs):
    x = th.tanh(self.lin1(inputs))
    x = inputs + self.lin2(x)
    return x

In [ ]:
model = create_model(Model2)
losses, min_loss, best_state = fit(model, epochs=2000, optimizer=th.optim.AdamW(model.parameters(), lr=0.01))
save_if_better(type(model), min_loss, best_state)
plot_loss(losses)

In [ ]:
compare_many(ref_model, model)

In [ ]:
class Model3(th.nn.Module):
  '''Similar to model2, it tries to fit the non-linearity using tanh.
     But this makes the nn more powerfull by adding extra weights on the last layer'''
  def __init__(self):
    super(Model3, self).__init__()
    self.lin1 = th.nn.Linear(2, 2)
    self.lin2 = th.nn.Linear(4, 2)
  
  def forward(self, inputs):
    x = th.tanh(self.lin1(inputs))
    cat = th.cat([inputs, x], dim=1)
    x = self.lin2(cat)
    return x

In [ ]:
model = create_model(Model3)
losses, min_loss, best_state = fit(model, epochs=2000, optimizer=th.optim.AdamW(model.parameters(), lr=1e-5))
save_if_better(type(model), min_loss, best_state)
plot_loss(losses)

In [ ]:
compare_many(ref_model, model)

In [ ]:
class Model4(th.nn.Module):
  '''Trying to train 2 separate linear models by splitting the path with a softmax'''
  def __init__(self):
    super(Model4, self).__init__()
    self.emb = th.nn.Linear(2, 2, bias = False)
    self.lina = th.nn.Linear(2, 2)
    self.linb = th.nn.Linear(2, 2)
  
  def forward(self, inputs):
    emb = th.softmax(self.emb(inputs), dim = 1)
    self.soft = emb
    return emb[:,0].view(-1, 1)*self.lina(inputs) + emb[:,1].view(-1, 1)*self.linb(inputs)


In [ ]:
model = create_model(Model4)
losses, min_loss, best_state = fit(model, epochs=2000, optimizer=th.optim.AdamW(model.parameters(), lr=1e-7))
save_if_better(type(model), min_loss, best_state)
plot_loss(losses)

In [ ]:
model4 = create_model(Model4)
compare_many(ref_model, model4)

In [ ]:
def plot_soft(model, kfn):
  '''Plot the softmax based on the mapping function kfn'''
  with th.no_grad():
    model(Xtrain)
    soft: th.Tensor = th.cat([Xtrain, model.soft], dim=1)

    soft_sum0 = map_reduce(soft, kfn, lambda acc, x: (acc[0]+1, acc[1]+x[2]) if acc else (1, x[2]))
    soft_sum1 = map_reduce(soft, kfn, lambda acc, x: (acc[0]+1, acc[1]+x[3]) if acc else (1, x[3]))
    
    def avg(s):
      r = {}
      for k, v in s.items():
        r[k] = v[1] / v[0]
      return r
    
    soft_sum0 = avg(soft_sum0)
    soft_sum1 = avg(soft_sum1)
    
    fig, (ax0, ax1) = plt.subplots(2)
    ax0.bar(soft_sum0.keys(), soft_sum0.values())
    ax1.bar(soft_sum1.keys(), soft_sum1.values())


# This transition should happen strongly at -50, but instead starts happening mildly around -40
plot_soft(model4, lambda x: int(x[1]))

In [ ]:
# The position is tricking the embedding somehow. We know that position should not affect how we change velocity,
# but the nn finds a way to correlate them
plot_soft(model4, lambda x: int(x[0]))

In [ ]:
class Model5(th.nn.Module):
  '''Trying to train 2 separate linear models by splitting the path with a softmax.
     The difference now is making the split stronger'''
  def __init__(self):
    super(Model5, self).__init__()
    self.emb = th.nn.Linear(2, 2)
    self.lina = th.nn.Linear(2, 2)
    self.linb = th.nn.Linear(2, 2)
  
  def forward(self, inputs):
    emb = th.softmax(self.emb(inputs), dim = 1)
    emb = emb**2
    emb /= emb.sum(dim=1, keepdim=True)
    self.soft = emb
    return emb[:,0].view(-1, 1)*self.lina(inputs) + emb[:,1].view(-1, 1)*self.linb(inputs)

In [ ]:
model = create_model(Model5)
losses, min_loss, best_state = fit(model, epochs=10000, optimizer=th.optim.AdamW(model.parameters(), lr=1e-7))
save_if_better(type(model), min_loss, best_state)
plot_loss(losses)

In [ ]:
compare_many(ref_model, model)

In [ ]:
model5 = create_model(Model5)

# Based on vel
plot_soft(model5,  lambda x: int(x[1]))

In [ ]:
# Based on pos
plot_soft(model5,  lambda x: int(x[0]))

In [ ]:
class Model6(th.nn.Module):
  '''Like model5, but lets try to improve the encoding. We know that the non linearity depends only on velocity, not position.
     So lets exploit that.'''
  def __init__(self):
    super(Model6, self).__init__()
    self.emb = th.nn.Linear(1, 2, bias=False)
    self.lina = th.nn.Linear(2, 2)
    self.linb = th.nn.Linear(2, 2)
  
  def forward(self, inputs):
    emb = self.emb(inputs[:, 1][:, None])
    soft = th.softmax(emb, dim = 1)
    self.soft = soft
    return soft[:,0].view(-1, 1)*self.lina(inputs) + soft[:,1].view(-1, 1)*self.linb(inputs)

In [ ]:
model = create_model(Model6, False)
losses, min_loss, best_state = fit(model, epochs=10000, optimizer=th.optim.AdamW(model.parameters(), lr=1e-1))
save_if_better(type(model), min_loss, best_state)
plot_loss(losses)

In [ ]:
# model = create_model(Model6)
compare_many(ref_model, model)

In [ ]:
model6 = create_model(Model6)
plot_soft(model6,  lambda x: int(x[1]))

In [ ]:
plot_soft(model6,  lambda x: int(x[0]))

In [ ]:
def plot_embs(model):
  with th.no_grad():
    model(Xtrain)
    # soft = model.soft.argmax(dim=1)
    # print('soft', soft.unique())

    print('model.soft:', model.soft.shape)
    soft: th.Tensor = th.cat([Xtrain, model.soft], dim=1)

    print('soft:', soft.shape)

    # def redfn(acc, x):
    #   # print(x.shape)
    #   return (acc if acc else 0)+x[2]

    soft_sum0 = map_reduce(soft, lambda x: int(x[1]), lambda acc, x: (acc if acc else 0)+x[2])
    soft_sum1 = map_reduce(soft, lambda x: int(x[1]), lambda acc, x: (acc if acc else 0)+x[3])
    # soft_sum_total = {}
    # def add(s):
    #   for k, v in s.items():
    #     if k not in soft_sum_total:
    #       soft_sum_total[k] = 0
    #     soft_sum_total[k] += v
    # add(soft_sum0)
    # add(soft_sum1)
    
    fig, (ax0, ax1) = plt.subplots(2)
    
    ax0.bar(soft_sum0.keys(), soft_sum0.values())
    
    ax1.bar(soft_sum1.keys(), soft_sum1.values())

    # h_l = 2
    # soft_h = th.zeros((2, maxv*2 // h_l + 1), requires_grad=False)
    # for i, x in enumerate(Xtrain):
    #   idx: int = int(x[1] / h_l) + maxv // h_l
    #   soft_h[soft[i]][idx] += 1


    # img = plt.plot(range(len(soft_h[1])), soft_h[1])
    # img = plt.plot(range(len(soft_h[0])), soft_h[0])

In [ ]:
print(saved_models)
for k, v in saved_models.items():
  filename = 'models/' + k.replace('<class \'__main__.', '').replace('\'>', '')
  print(filename)
  th.save(v, filename)

In [ ]:
import os
smodels = {}
for f in os.listdir('models'):
  smodels[f] = th.load('models/'+f)

print(smodels)

In [ ]:
class ModelPos(th.nn.Module):
  '''Trying to train 2 separate linear models by splitting the path with a softmax just for pos'''
  def __init__(self):
    super(ModelPos, self).__init__()
    # self.emb = th.nn.Linear(2, 2, bias = False)
    self.lina = th.nn.Linear(2, 1)
    # self.linb = th.nn.Linear(2, 1)
  
  def forward(self, inputs):
    # emb = th.softmax(self.emb(inputs), dim = 1)
    # self.soft = emb
    # return emb[:,0].view(-1, 1)*self.lina(inputs) + emb[:,1].view(-1, 1)*self.linb(inputs)
    return self.lina(inputs)

In [ ]:
model = create_model(ModelPos)
losses, min_loss, best_state = fit(model, Y=Ytrain[:, 0][:, None], epochs=1000, optimizer=th.optim.AdamW(model.parameters(), lr=1e-5))
save_if_better(type(model), min_loss, best_state)
plot_loss(losses)

In [ ]:
class ModelVel2(th.nn.Module):
  '''Trying to train 2 separate linear models by splitting the path with a softmax just for vel'''
  def __init__(self):
    super(ModelVel2, self).__init__()
    # self.query = th.nn.Linear(1, 2, bias = False)
    # self.value = th.nn.Linear(, 1, bias = False)
    self.lina = th.nn.Linear(1, 1)  
    self.linb = th.nn.Linear(1, 1)
    # self.dca = th.nn.Parameter(th.rand(1))
    # self.dcb = th.nn.Parameter(th.rand(1))
  
  def forward(self, inputs):
    # print(inputs.shape)
    # q = self.query(inputs[:, 1:2])
    # w = th.softmax(q, dim = -1)
    # self.soft = w
    # # print(w)
    # # v = th.cat([self.lina(inputs), self.linb(inputs)], dim=1)
    # v = th.cat([inputs[:, 1:2] - self.dca, inputs[:, 1:2] - self.dcb], dim=1)
    # arg = th.argmax(w, dim=1)
    # # print(v.shape)
    # # print(v.view(-1, 2, 1))
    # # result = w.view(-1, 1, 2) @ v.view(-1, 2, 1)
    # result = arg
    # print('result', result)
    # return result.view(-1, 1)
    vel = inputs[:, 1:2]
    w0 = (vel < -50).type(th.float32)
    w1 = 1 - w0
    w = th.cat([w0, w1], dim=1)
    # print(w)
    # print(w.shape)
    v = th.cat([self.lina(vel), self.linb(vel)], dim=1)
    # print(v.shape)
    result = w.view(-1, 1, 2) @ v.view(-1, 2, 1)
    return result.view(-1, 1)
    

In [ ]:
# del saved_models[str(ModelVel2)]
model = create_model(ModelVel2, load_state=True)
losses, min_loss, best_state = fit(model, Y=Ytrain[:, 1][:, None], batch_size=None, epochs=10000, optimizer=th.optim.AdamW(model.parameters(), lr=1e-2))
save_if_better(type(model), min_loss, best_state)
plot_loss(losses)


In [ ]:
# print(model.state_dict())
for k,v in model.state_dict().items():
  print(k)
  print('  ',v)

In [ ]:
class ModelVelPos(th.nn.Module):
  def __init__(self):
    super(ModelVelPos, self).__init__()
    self.pos = create_model(ModelPos)
    self.vel = create_model(ModelVel2)
  
  def forward(self, inputs):
    p = self.pos(inputs)
    v = self.vel(inputs)
    r = th.cat([p, v], dim=1)
    # print(r.shape)
    return th.cat([p, v], dim=1)

In [ ]:
model = ModelVelPos()
compare_many(ref_model, model)